In [220]:
import pandas as pd
import parser
import unicodedata
import imp
import cyrtranslit
from spacy.tokenizer import Tokenizer
from spacy.lang.ru import Russian
pd.options.display.max_rows = 999
imp.reload(parser)

<module 'parser' from '/home/ubuntu/PycharmProjects/nko_classification/parser.py'>

In [187]:
dataset = pd.read_excel("np_ngo_classification.xlsx")
dataset.head(1)

,Регион,Название организации,ИНН,Год регистрации,Адресс,Возможность пожертвовать деньги на сайте,Возможность обратиться за помощью,Ссылка на сайт,Ссылка на соц сеть,Вид деятельности 1,...,Вид деятельности 3,Вид деятельности 4,Вид деятельности 5,Вид деятельности 6,Вид деятельности 7,Вид деятельности 8,Вид деятельности 9,Вид деятельности 10,Вид деятельности 11,Вид деятельности 12
0,Москва,Межрегиональная общественная благотворительная...,5032173157,2007-07-12,"г Москва, Орехово-Борисово Южное р-н, МКАД 23-...",0,0,"http://greenstartup.ru/, http:/9911850.ru/",https://princip1.livejournal.com/,3_2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [188]:
sites = dataset[~dataset["Ссылка на сайт"].isnull()]
sites.drop(columns=["Ссылка на соц сеть"], inplace=True)
social = dataset[~dataset["Ссылка на соц сеть"].isnull()]
social["Ссылка на сайт"] = social["Ссылка на соц сеть"]
social.drop(columns=["Ссылка на соц сеть"], inplace=True)
concat_dataset = pd.concat([sites, social], ignore_index=True)

/home/ubuntu/.virtualenvs/jup_env/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [189]:
def site_splitter(url_string):
    result = []
    urls = []
    for url in url_string.split(","):
        urls.append(url)
    for url in urls:
        for each in url.split("\n"):
            result.append(each)
    return result

concat_dataset["Ссылка на сайт"] = concat_dataset["Ссылка на сайт"].apply(site_splitter)
dataset.dropna(axis=0, subset=["Ссылка на сайт"], inplace=True)
concat_dataset["num_sites"] = concat_dataset["Ссылка на сайт"].apply(len)

In [192]:
# TODO: 3 times-loop
max_len = len(concat_dataset)
for index, row in concat_dataset.iterrows():
    if len(row["Ссылка на сайт"])>1:
        sites = row["Ссылка на сайт"]
        last_site = sites.pop(-1)
        concat_dataset["Ссылка на сайт"].loc[index] = sites
        row["Ссылка на сайт"] = [last_site]
        concat_dataset.loc[max_len] = row
        max_len += 1

In [193]:
concat_dataset["Ссылка на сайт"] = concat_dataset["Ссылка на сайт"].apply(lambda lst: lst[0].strip())

In [197]:
# Adding http prefix 
def fix_url(url):
    if not isinstance(url, str):
        return None
    if not url.startswith("http"):
        return "http://"+url
    return url


concat_dataset["Ссылка на сайт"] = concat_dataset["Ссылка на сайт"].apply(fix_url)

In [200]:
concat_dataset

,Регион,Название организации,ИНН,Год регистрации,Адресс,Возможность пожертвовать деньги на сайте,Возможность обратиться за помощью,Ссылка на сайт,Вид деятельности 1,Вид деятельности 2,...,Вид деятельности 4,Вид деятельности 5,Вид деятельности 6,Вид деятельности 7,Вид деятельности 8,Вид деятельности 9,Вид деятельности 10,Вид деятельности 11,Вид деятельности 12,num_sites
0,Москва,Межрегиональная общественная благотворительная...,5032173157,2007-07-12,"г Москва, Орехово-Борисово Южное р-н, МКАД 23-...",0,0,http://greenstartup.ru/,3_2,6_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
1,Москва,Межрегиональная благотворительная общественная...,7702370774,2007-08-28,"г Москва, Мещанский р-н, ул Трифоновская, д 49...",1,1,http://oppid.ru/,2_1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,Москва,Некоммерческая организация Благотворительный ф...,7702472455,2014-02-11,"г Москва, Мещанский р-н, ул Щепкина, д 9 стр 1",1,1,http://capital-mercy.ru,1_9,1_2,...,2_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,Москва,Российский общественный благотворительный фонд...,7704058560,1998-06-05,"г Москва, р-н Соколиная Гора, ул Щербаковская,...",1,1,http://www.rbfondveteranov.ru/,1_4,5_2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,Москва,Благотворительный фонд поддержки и реализации ...,7704278848,2011-09-30,"г Москва, р-н Хамовники, пер Знаменский Б., д ...",0,0,http://www.avccharity.com/,4_2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
5,Москва,"Благотворительный фонд ""Фонд Валерия Гергиева""",7709402778,2003-01-27,"г Москва, р-н Арбат, ул Молчановка Б., д 23 к 2",0,0,http://gergiev-charity.com/rus/,4_2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
6,Москва,Международный Благотворительный Фонд Владимира...,7710012370,1994-05-20,"г Москва, Пресненский р-н, Шведский тупик, ...",1,0,http://spivakov.ru/fund/contacts/,4_2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
7,Москва,Благотворительный фонд поддержки детей с особе...,7710479817,2012-05-02,"г Москва, Пресненский р-н, пер Кондратьевский ...",1,1,http://yaest.ru/,1_7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
8,Москва,Благотворительный фонд помощи бездомным животн...,7716451494,2014-11-14,"г Москва, Ярославский р-н, Ярославское шоссе, ...",1,0,http://fond-kovcheg.ru,6_4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9,Москва,Благотворительный фонд поддержки детей имени и...,7701064809,2011-03-22,"105082, ГОРОД МОСКВА, УЛИЦА ФРИДРИХА ЭНГЕЛЬСА,...",1,1,http://www.vfonde.ru/,1_7,1_2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [216]:
parser.get_filtered_text("http://spivakov.ru/fund/contacts/")

'105187, Москва, Щербаковская ул., 50/52 Главная Персоналии Российскому фонду ветеранов 25 лет Акции Российского фонда ветеранов Постановления о материальной помощи в 2017году. Контакты «Туризм для граждан старшего поколения» По направлению Российского фонда ветеранов за поступившие средства от спонсоров в Герантологическом центре «Монино» прошли курс реабилитации ветераны и участники ВОВ Новости В ВОРОНЕЖЕ ВОЕННЫЕ ПИСАТЕЛИ ПОДВЕЛИ ИТОГИ 2018 ГОДА Заседание Воронежской областной общественной организации «Союз военных писателей» (СВП) прошло 3 ноября 2018 года. На нём были подведены итоги работы в 2018 году. ИЗВЕСТНОЙ ВОРОНЕЖСКОЙ ПОЭТЕССЕ – 70 ЛЕТ Сотрудники регионального отделения Российского фонда ветеранов совместно с членами Воронежской областной общественной организации «Союз военных писателей» чествовали ветерана труда, известную поэтессу. «РУСИЧИ-ЦЕНТР» выдал Российскому фонду ветеранов Золотой сертификат благотворителя, свидетельствующий о том, что Фонд принял в 2018 году активн

In [ ]:
concat_dataset["web_text"] = concat_dataset["Ссылка на сайт"].apply(parser.get_filtered_text)

In [219]:
concat_dataset["web_text"].loc[22]

'105187, Москва, Щербаковская ул., 50/52 Главная Персоналии Российскому фонду ветеранов 25 лет Акции Российского фонда ветеранов Постановления о материальной помощи в 2017году. Контакты «Туризм для граждан старшего поколения» По направлению Российского фонда ветеранов за поступившие средства от спонсоров в Герантологическом центре «Монино» прошли курс реабилитации ветераны и участники ВОВ Новости В ВОРОНЕЖЕ ВОЕННЫЕ ПИСАТЕЛИ ПОДВЕЛИ ИТОГИ 2018 ГОДА Заседание Воронежской областной общественной организации «Союз военных писателей» (СВП) прошло 3 ноября 2018 года. На нём были подведены итоги работы в 2018 году. ИЗВЕСТНОЙ ВОРОНЕЖСКОЙ ПОЭТЕССЕ – 70 ЛЕТ Сотрудники регионального отделения Российского фонда ветеранов совместно с членами Воронежской областной общественной организации «Союз военных писателей» чествовали ветерана труда, известную поэтессу. «РУСИЧИ-ЦЕНТР» выдал Российскому фонду ветеранов Золотой сертификат благотворителя, свидетельствующий о том, что Фонд принял в 2018 году активн

In [6]:
mapping = pd.read_excel("np_ngo_mappings.xlsx")
mapping

In [29]:
filtered = [e for e in text_list if detect(e)=="ru"]
filtered

['О центре - Иппоцентр Поводья Жизни. Крым, Алушта.',
 'О центре',
 'Наш коллектив',
 'Наши воспитанники',
 'Новости',
 'Отзывы',
 'Контакты',
 'Вы здесь:',
 'Главная',
 'Искать...',
 'Иппоцентр "Поводья жизни"',
 'О центре',
 'Наш коллектив',
 'Наши воспитанники',
 'Новости',
 'Отзывы',
 'Контакты',
 'Бесплатные шаблоны',
 'О центре',
 'Наш центр ожил и начал свое существование с апреля 2007 года, с появления первой нашей маленькой пациентки. В июне 2007 года была зарегистрирована благотворительная организация - Иппотерапевтический центр «Поводья жизни». И мечта о помощи особенным деткам о том, чтоб подарить им возможность общения с прекрасными животными, стала реальностью. Ведь польза, приносимая лечебной верховой ездой, уже давно признана во всем мире работниками медицины, образования и социальной сферы.',
 'Здесь на Южном берегу Крыма с уникальной природой и климатом есть все возможности дарить детям здоровье и желание полноценно жить несмотря ни на что.',
 'Дети, которые попадают 

In [12]:
dataset["web_text"] = dataset["Ссылка на сайт"].apply(parser.get_text_contents)

/home/ubuntu/PycharmProjects/nko_classification/parser.py:33: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 33 of the file /home/ubuntu/PycharmProjects/nko_classification/parser.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  data = soup.findAll(text=True)


KeyboardInterrupt: 

In [7]:
dataset["links"] = dataset["Ссылка на сайт"].apply(parser.get_links)

/home/ubuntu/PycharmProjects/nko_classification/parser.py:16: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 16 of the file /home/ubuntu/PycharmProjects/nko_classification/parser.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(html)


In [182]:
dataset

,Регион,Название организации,ИНН,Год регистрации,Адресс,Возможность пожертвовать деньги на сайте,Возможность обратиться за помощью,Ссылка на сайт,Ссылка на соц сеть,Вид деятельности 1,...,Вид деятельности 4,Вид деятельности 5,Вид деятельности 6,Вид деятельности 7,Вид деятельности 8,Вид деятельности 9,Вид деятельности 10,Вид деятельности 11,Вид деятельности 12,links
0,Москва,Межрегиональная общественная благотворительная...,5032173157,2007-07-12,"г Москва, Орехово-Борисово Южное р-н, МКАД 23-...",0,0,http://greenstartup.ru/,https://princip1.livejournal.com/,3_2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
1,Москва,Межрегиональная благотворительная общественная...,7702370774,2007-08-28,"г Москва, Мещанский р-н, ул Трифоновская, д 49...",1,1,http://oppid.ru/,https://www.facebook.com/oppid.ru/,2_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[http://oppid.ru, http://oppid.ru/, http://opp..."
2,Москва,Некоммерческая организация Благотворительный ф...,7702472455,2014-02-11,"г Москва, Мещанский р-н, ул Щепкина, д 9 стр 1",1,1,http://capital-mercy.ru,https://www.facebook.com/capitalmercy,1_9,...,2_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[http://capital-mercy.ru, http://capital-mercy..."
3,Москва,Российский общественный благотворительный фонд...,7704058560,1998-06-05,"г Москва, р-н Соколиная Гора, ул Щербаковская,...",1,1,http://www.rbfondveteranov.ru/,None,1_4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[http://www.rbfondveteranov.ru/, http://www.rb..."
4,Москва,Благотворительный фонд поддержки и реализации ...,7704278848,2011-09-30,"г Москва, р-н Хамовники, пер Знаменский Б., д ...",0,0,http://www.avccharity.com/,None,4_2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
5,Москва,"Благотворительный фонд ""Фонд Валерия Гергиева""",7709402778,2003-01-27,"г Москва, р-н Арбат, ул Молчановка Б., д 23 к 2",0,0,http://gergiev-charity.com/rus/,None,4_2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
6,Москва,Международный Благотворительный Фонд Владимира...,7710012370,1994-05-20,"г Москва, Пресненский р-н, Шведский тупик, ...",1,0,http://spivakov.ru/fund/contacts/,None,4_2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[http://spivakov.ru/fund/, http://spivakov.ru/..."
7,Москва,Благотворительный фонд поддержки детей с особе...,7710479817,2012-05-02,"г Москва, Пресненский р-н, пер Кондратьевский ...",1,1,http://yaest.ru/,https://vk.com/fond_ya_est\nhttps://www.facebo...,1_7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
8,Москва,Благотворительный фонд помощи бездомным животн...,7716451494,2014-11-14,"г Москва, Ярославский р-н, Ярославское шоссе, ...",1,0,http://fond-kovcheg.ru,"https://vk.com/public79857356, https://www.fac...",6_4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[http://fond-kovcheg.ru/login, http://fond-kov..."
9,Москва,Благотворительный фонд поддержки детей имени и...,7701064809,2011-03-22,"105082, ГОРОД МОСКВА, УЛИЦА ФРИДРИХА ЭНГЕЛЬСА,...",1,1,http://www.vfonde.ru/,None,1_7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[http://www.altland.ru, http://www.vfonde.ru/,..."


In [40]:
test_url = 'http://oppid.ru/zasedanie-rabochey-gruppyi-po-redkim-zabolevaniyam-pod-predsedatelstvom-valentinyi-ivanovnyi-shirokovoy/'

In [60]:
tr = cyrtranslit.to_cyrillic('http://oppid.ru/zasedanie-rabochey-gruppyi-po-redkim-zabolevaniyam-pod-predsedatelstvom-valentinyi-ivanovnyi-shirokovoy/', 'ru')

In [53]:
nlp = Russian()
# nlp = spacy.load('xx_ent_wiki_sm')
tokenizer = Tokenizer(nlp.vocab)

In [62]:
doc = nlp(tr)
print([token.text for token in doc])

['хттп://оппид.ру', '/', 'заседание', '-', 'рабочей', '-', 'группыи', '-', 'по', '-', 'редким', '-', 'заболеваниям', '-', 'под', '-', 'председателством', '-', 'валентиныи', '-', 'ивановныи', '-', 'широковой/']


In [32]:
parser.get_links(dataset["Ссылка на сайт"].loc[1])

['http://oppid.ru/nashi-tseli/',
 'http://oppid.ru/kontaktyi/',
 'http://oppid.ru/zasedanie-rabochey-gruppyi-po-redkim-zabolevaniyam-pod-predsedatelstvom-valentinyi-ivanovnyi-shirokovoy/',
 'http://oppid.ru/articles/6-different-primary-immunodeficiencies/',
 'http://oppid.ru/forum/memberlist.php?mode=viewprofile&u=2',
 'http://oppid.ru',
 'http://oppid.ru/vopros-doktoru/',
 'http://oppid.ru/25-fevralya-2015-otkryitoe-zasedanie-soveta-obshhestvennyih-organizatsiy-po-zashhite-prav-patsientov-pri-ministerstve-zdravoohraneniya-rossiyskoy-federatsii/',
 'http://oppid.ru/rabota-nado-perechnem-redkih-zabolevaniy/',
 'http://oppid.ru/o-nas/stat-chlenom-oppid/',
 'http://oppid.ru/o-nas/pozhertvovat/',
 'http://oppid.ru/o-nas/',
 'http://oppid.ru/forum/viewtopic.php?t=7',
 'http://oppid.ru/donoram/',
 'http://oppid.ru/o-nas/druzhestvennyie-organizatsii/',
 'http://oppid.ru/forum/viewtopic.php?t=6',
 'http://oppid.ru/informatsionnyiy-kanal/',
 'http://oppid.ru/']

In [13]:
imp.reload(parser)
lnk = dataset["Ссылка на сайт"].loc[1]
parser.get_text_list(dataset["Ссылка на сайт"].loc[1])

/home/ubuntu/PycharmProjects/nko_classification/parser.py:16: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 16 of the file /home/ubuntu/PycharmProjects/nko_classification/parser.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(html)


['html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd"',
 'ОППИД | Общество пациентов с первичным иммунодефицитом',
 '\n\n a[href$=".pdf"], a[href$=".txt"], a[href$=".mp3"], a[href$=".aiff"], a[href$=".wav"], a[href$=".ogg"], a[href$=".wma"], a[href$=".zip"], a[href$=".rar"], a[href$=".ace"], a[href$=".tar.gz"], a[href$=".tgz"], a[href$=".mpg"], a[href$=".mpeg"], a[href$=".avi"], a[href$=".mv2"], a[href$=".mov"], a[href$=".mp4"], a[href$=".wmv"], a[href$=".swf"], a[href$=".xls"], a[href$=".doc"], a[href*="trackback"], a[href*="@"], a[href*="rss2"], a[href*="feed"], a[href*="postcomment"], a[href*="respond"], a[href*="action=edit"]\n{\n\t\tpadding-bottom:2px;\n\t\tpadding-left:22px;\n\t\tpadding-top:2px;\n\t\tbackground:transparent;\n\t}\n a[href$=".pdf"]\n\t\t{\n\t\t\tbackground: url(http://oppid.ru/wp-content/plugins/file-icons/icons/silkicons/page_white_acrobat.png) no-repeat scroll left center;\n\t\t}\n a[href$=".txt"]\n\t\t{

In [21]:
unicodedata.normalize("NFD", '© Общество пациентов с первичными иммунодефицитами')

'© Общество пациентов с первичными иммунодефицитами'